## Feynman Exploring Toronto

This is the notebook for the exploration of Toronto neighbourhood

In [1]:
#import required libraries
import numpy as np # library to handle data in a vectorized manner
import requests 
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!conda install -c conda-forge bs4 --yes
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge lxml --yes

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from bs4 import BeautifulSoup
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## define the URL and scrape request

In [2]:

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)#.json()


## check scrape status

In [3]:
results.status_code
#np.save("results.txt",results.text)

200

In [4]:
type(results.text)

str

## Use Beautiful Soup Libirary to format the response

In [17]:
soup = BeautifulSoup(results.content, 'lxml')
print (type(soup))

<class 'bs4.BeautifulSoup'>


In [48]:
import bs4


In [127]:
count = 0
dic = {}
list1 = []

for i in soup.tbody.children:
    # check if the repsonse is a bs4 tag
    if isinstance(i, bs4.element.Tag):
        
        #split the title versus the content body
        if count == 0:
            for titles in i.children:
                
                # check if the repsonse is a bs4 tag
                if isinstance(titles, bs4.element.Tag):
                    
                    #define the dictionary keys with the titles, and remove "\n"
                    dic[str(titles.contents[0]).replace('\n','')] = ''
        else:
            num = 0
            a =[0,1,2,0] 
            for bodies in i.children:
                if isinstance(bodies, bs4.element.Tag):
                    num += 1
                    
                    #define the content with the response body, and remove "\n"
                    a[num] = bodies.contents[0].replace('\n','')
            list1.append(a)
        count += 1

list_po1 = []
list_po2 = []
list_po3 = []

# append the content to the three columns
for i in range(np.shape(list1)[0]):
    list_po1.append(list1[i][1])
    list_po2.append(list1[i][2])
    list_po3.append(list1[i][3])


# match the keys with the contents
dic['Postal Code']  = list_po1
dic['Borough']  = list_po2
dic['Neighborhood']  = list_po3

#transfer the dictionary to pandas dataframe
data1 = pd.DataFrame(dic)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## data cleaning

In [135]:
#remove rows with Borough not assigned
data2 = data1[data1['Borough'] != 'Not assigned']
data2.head(8)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills


In [139]:
# print out the shape
data2.shape

(103, 3)